1) Объясните смысл и напишите формулы для следующих метрик производительности процессора: IPC, CPI, Performance, Dynamic Power

- IPC, instructions per clock - среднее количество инструкций, которое может исполнить процессор за один такт.
$$ IPC = \dfrac{N_{machine \space instructions \space in \space program}}{N_{clocks \space elapsed}} $$

- CPI, clocks per instruction - среднее количество тактов, которое процессор тратит на исполнение инструкции.
$$ PCI = \dfrac{1}{PCI} $$

- Performance, производительность - скорость с которой мы исполняем программу. Одна из трех основных метрик, которые лежат в основе проектирования процессора - performance, power, cost
$$ Performance = \dfrac{1}{Time} = \dfrac{1}{N_{cycles} T_{per \space cycle}} = \dfrac{1}{N_{instr} T_{per \space cycle} CPI} = \dfrac{ f \cdot IPC }{N_{instr}} $$

- Dynamic Power, динамическая потребляемая мощность - затраты энергии на перезарядку емкостей при переключении транзистров в процессоре.
$$ DynPower = \dfrac{1}{2}\cdot C_{eff} \cdot V_{dd}^2 \cdot a \cdot f $$
Где $a$ - это частота переключения, раз в сколько тактов в среднем переключается транзистор

2) Что такое суперскалярный (superscalar) процессор?

Суперскалярный процессор - процессор, способный обрабатывать 2 и более инструкции за цикл, используя дополнительные пайплайны или альтернативные методы, такие как VLIW, векторизация

3) Какие типы зависимостей по данным существуют? Приведите примеры аппаратных оптимизаций, которые позволяют сократить связанные задержки или разрешить каждый тип зависимостей.

- Anti-dependency: write after read
- Output dependency: write after write
- True Dependency: read after write

Первые два вида зависимостей не являются настоящими зависимостями, т.к. они существуют только из-за недостатка имен в ISA (по большому счету мы ждем, пока все consumers прочитают source перед тем, как его перезаписать, хотя могли бы просто положить source в другое место, вместо перезаписи). Поэтому основным методом борьбы с такими зависимостями является register renaming. Суть register renaming состоит в следующем: при попадании инструкции в SQ (scheduler queue) или ROB (при отсутствии SQ) производится реаллокация регистров с целью удалить все false dependencies, при этом формируется таблица соответствия между isa register и physical register (RAT - reqister alias table). Еще один способ борьбы с false dependency - это разбиение операций store на ве - вычисление адреса (STA) и данных (STD), но об этом в следующем вопросе

True dependency нельзя удалить таким способом, т.к. операции store не поддаются спекулятивному исполнению и не могут быть переорганизованы меджу собой. Первой попыткой улучшить производительность на таких зависимостях - store forwarding. Суть store frowarding состоит в следующем - load инструкции берут данные из producer store напрямую, а не через кэш. Для этого требуется только вычисление адреса и размера store и load. Если они совпадают, то store forwarding возможен. Таким образом мы ждем только адрес (для store и load), данные (для sotre), и очередь при исполнении (т.к. store нельзя исполнять спекулятивно или out of order относительно других store, а для load таких ограничений нет). Понять, есть пересечение по данным или нет помогает Load/Store Buffer, где хранятся вычисленные адреса предыдущих store и их статус. По этим данным load может сразу понять, есть у него зависимость от предыдущего store, и является ли она forwardable.

4) С какой целью инструкцию Store разделяют на микро-операции STA (Store address calculation) и STD (Store data calculation)?

STA и STD позволяют удалить false dependency при read after write. В классической реализации, где store - это одна инструкция, мы не узнаем адреса, по которму производится запись до момента, пока не будет исполнена вся инструкция, тоесть пока не будут разрезолвлены адрес и данные. Однако последующим load инструкциям в первую очередь нужно знать об адресе записи, чтобы понять, есть пересечение по данным или нет. Таким образом получается, что load не будет исполнен пока не будут получены адрес ис данные. Чтобы избежать этой ненужной зависимости, вычисление данных и адреса разбивается на две инструкции. Благодаря этому load ждет данных только если есть пересечение по ним, т.к. у него есть возможность заранее сравнить адреса для записи и чтения.

5) Объясните назначение и функции следующих аппаратных структур: ROB, Scheduler Queue (Issue Queue, Reservation Station), RAT, PRF, Load Buffer, Store Buffer
- ROB, re-order buffer - решение для поддержания корректного архитектурного состояние при out-of-order исполнении. Мы заводим буфер на определенное количество инструкций, которые исполняем спекулятивно и out-of-order, где это возможно в виду отсутсвия зависимостей (методы предотвращения и улучшения зависимостей, а также их проверки описаны в предыдущих вопросах). Инструкции в ROB изменяют только локальное спекулятивное состояние, не затрагивая архитектурное. Поддержание архитектурного состояния происходит следующим образом - инструкция покидает ROB и изменяет состояние только когда она исполнена и является самой старой в буфере. Таким образом поддерживается внешняя видимость in-order исполнения. Размер ROB ограничен слежующими факторами - количество бранчей (штрафом за их миспредикт) и ложные зависимости (чем больше ROB, тем больше тратится времени на register renaming и поиск в RAT, падает эффективность этих техник). Также размер ROB ограничивает то, что каждый цикл мы должны проверять готовность каждой инструкции, чтобы продвинуть очередь исполнения зависимых инструкций. При большом ROB это занимает много времени и портит производительность.
- SQ, scheduler queue - решение обозначенной выше проблемы. Мы выносим из ROB все инструкции, которые еще не закончили исполнение, тем самым сокращая обход при приверке готовности в несколько раз. Все проверки и описанные выше алгоритмы (аллокация, реаллокация и scheduling) теперь нужно проводить лишь на этой малой очереди, вынесенной из ROB. Сам же ROB используется только для восстановления порядка исполнения при retiring.
- RAT, register alias table - таблица соответствия между isa регистрами и физическими регистрами, полученнная при register renaming. Каждый ROB хранит предыдущую RAT, и обновляет ее при обновлении. 
- PRF, physical register file - регистровый файл, содержащий маппинг физических регистров процессора. Используется в том числе во время реаллокации регистров при register renaming, во время которого каждому isa register динамически ставится в соответствие какой либо physical register. После чего это соответствие используется при спекулятивном исполнении и последующем обновлении architectural state при retirement.
- Load Buffer, Store Buffer - аналогично SQ, Load Buffer - это подмножество ROB, в котором хранятся только операции load и необходимая для них информация, такая как: адрес, пердыдущий store, readiness, и статус, ожидает load выполнения store или нет. Наличие Load Buffer помогает быстрее разрешать false dependency, т.к. уменьшается количество инструкций, которые необходимо обходить. Load Buffer работает сообща с Store buffer - структурой в процессоре, в которой хранятся вычисленные адреса исполняемых store инструкций и данные для загрузки (Store Buffer позволяет делать спекулятивное исполнение Store инструкции, т.к. сначала store идет именно в него, не влияя на кэш и память. Однако при обновлении ROB они будут не спекулятивно перенесены в память, и их уже нельзя будет откатить). Эта таблица позволяет избежать false denendency load инструкции от store, т.к. есть возможность заранее сравнить адреса.

6) Пусть каждая 5ая инструкция в процессоре это Branch. Предсказатель переходов имеет точность 90%. Оцените, ROB какого максимального размера имеет смысл для такого процессора

При миспредикте бранч инструкции происходит удалени всех последующих инструкций из ROB, что приводит к его неэффективному использованию. Мы ходим избежать подобной ситуации. Тогда требуется оченить вероятность $P_N$ для ROB размером $N$ того, что все $N$ инструкций будут исполнены, и не будут выкинуты. В среднем, каждая пятая инструкция - это бранч. тогда:

$$ P_{branch} = 0.2 $$
$$ P_{predict} = 0.9 $$
$$ N_{branch} = N \cdot P_{branch} $$
$$ P_N = P_{predict}^{N_{branch}} = P_{predict}^{N \cdot P_{branch} } $$
$$ \ln{P_N} = \ln{P_{predict}} \cdot N \cdot P_{branch} $$
$$ N = \dfrac{\ln{P_N}}{\ln{P_{predict}} \cdot P_{branch}} $$

Пусть удовлетворимая вероятность того, что ROB не будет смыт - 95%. Тогда:

$$ N = \dfrac{\ln{0.95}}{\ln{0.9} \cdot 0.2} \approx 2.43 $$

Для 90%:

$$ N = \dfrac{\ln{0.9}}{\ln{0.9} \cdot 0.2} = 5 $$

Таким образом получаем, что с такой точностью branch predictor делать ROB вообще не имеет смысла, т.к. он имел бы длину 5 и сбрасывался бы каждый 10й цикл


7) Что такое Memory Disambiguation?

Memory Disambiguation - это техники, применяемые OOO процессорами для того, чтобы разрешить зависимости по данным в программе. Эти техники находят true dependencies, убирают false dependencies, что позволяет улучшить ILP за счет безопасного OOO исполнения load/store. Эти техники включают в себя описанные выше store forwarding, register renaming и другие


8) В чем заключается проблема со спекулятивным исполнением Store инструкций?

Проблема со спекулятивным исполнением Store инструкций заключается в необходимости поддерживать относительный порядок store инструкций, а также с отсутствием механизма отката изменений. Без введения структур типа Store Buffer такого достичь невозможно, и то, он помогает только с тем, чтобы не портить arch state, но проблема невозможности отката одного или нескольких store все еще остается актуальной.

9) Что такое Store forwarding и Load speculation в OOO процессоре?

Store Forwarding был описан выше. Load Speculation - это техники, позволяющие исполнить load инструкции без исполнения всех предыдущих инструкций (без однозначного резолва load adress). Load Speculation включает такие техники, как Dependence Prediction, Address Prediction, Value Prediction, и Memory Renaming.

Dependence Prediction - предсказывает aliasing между load и store инструкциями. Dependence Prediction может предсказать либо что load не зависит ни от какого из предыдущих store, либо предсказать, от какого store данны load зависит. Эта спекуляция позволяет спекулятивно исполнить load без необходимости ожидания вычисления эффективных адресов всех предыдущих store.

Address Prediction - предсказывает адрес, по которому должен произвестись load. Это позволяет убрать зависимость load инструкции от непосредственного вычисления effective load adress, сделать data prefetching, а также ускорить поиск alias среди store инструкций.

Value Prediction - предсказывает данные, которые будут извлечены из памяти, что позволяет дальнейшее спекулятивное исполнение инструкций, зависящих от load.

Memory Renaming - предсказывает зависимости между laod и store и делает forward значений непосредственно из store то load через регистры или кэш, не производя запись в память. Memory renaming описается на наблюдение, что многие load обычно являются alias одного и того же store.

10) Что такое Simultaneous Multithreading?

Simultaneous Multithreading - это техника, позволяющая одновременное исполнение двух логических потоков на одном физическом ядре для увеличения эффективности исполнения, и для более равномерной нагрузки на аппаратуру. Этот эффект достигается разделением каждого из ядер на два логических, каждй со своим архитектурным состоянием (pc, regfile). При SMT ресурсы процессора делятся на replicated, partitioned и shared. Благодаря тому, что virtual threads делят одни и те же физические регистры (shared PRF), на стадии исполнения в SQ может исполнять их одновременно, не иммея понятия о том, к какому thread относится эта инструкция, что позволяет заполнять очереди более равномерно и стабильно, увеличивая общую производительность (однако чтобы поддерживать in-order retirement, ROB у виртуальных тредов раздельные).